In [2]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import scipy.sparse as sp
import numpy as np

In [3]:
brews_df = pd.read_pickle('data/brews_with_costs.pkl')
ingredients_df = pd.read_pickle('data/ingredients.pkl')
brews_df

,first_ingredient,second_ingredient,third_ingredient,effects,name,value,descriptions
0,Abecean Longfin,Ancestor Moth Wing,Cyrodilic Spadetail,"[Fortify Restoration, Damage Stamina]",Potion of Fortify Restoration,243,"[Drain the target's Stamina by 22 points., Res..."
1,Abecean Longfin,Ash Creep Cluster,Beehive Husk,"[Fortify Sneak, Fortify Destruction]",Potion of Fortify Destruction,432,[Destruction spells are 46% stronger for 60 se...
2,Abecean Longfin,Ash Creep Cluster,Chaurus Eggs,"[Weakness to Poison, Invisibility]",Potion of Invisibility,486,"[Invisibility for 37 seconds., Target is 15% w..."
3,Abecean Longfin,Ash Creep Cluster,Cyrodilic Spadetail,"[Fortify Restoration, Damage Stamina]",Potion of Fortify Restoration,243,"[Drain the target's Stamina by 22 points., Res..."
4,Abecean Longfin,Ash Creep Cluster,Elves Ear,"[Weakness to Frost, Resist Fire]",Potion of Resist Fire,190,"[Resist 28% of fire damage for 60 seconds., Ta..."
...,...,...,...,...,...,...,...
25237,Tundra Cotton,Void Salts,NA,"[Resist Magic, Fortify Magicka]",Potion of Fortify Magicka,194,[Magicka is increased by 37 points for 60 seco...
25238,Tundra Cotton,Wisp Wrappings,NA,[Resist Magic],Potion of Resist Magic,80,[Resist 9% of magic for 60 seconds.]
25239,Vampire Dust,White Cap,NA,[Restore Magicka],Potion of Restore Magicka,52,[Restore 58 points of Magicka.]
25240,Void Salts,Wisp Wrappings,NA,[Resist Magic],Potion of Resist Magic,80,[Resist 9% of magic for 60 seconds.]


In [4]:
brews = list(brews_df.name)
ingredients = list(ingredients_df.name)

The model is the following:

#### Parameters

$B = \{1, \dots, 25242\}$: Set of brews

$I = \{1, \dots, 109\}$: Set of ingredients

$I_p \subset{I}$: Set of ingredients required to make one unit of brew $b$ (note that $|I_b| \leq 3 \,\,\forall b \in B$)

$z_i \in \mathbb{Z}^+$: Amount of ingredient $i$ available

$c_b \in \mathbb{R}^+$: gold value of brew $b$

#### Decision Variables

$y_b \in \mathbb{Z}^+$: Number of brews $b$ created

$x_{ib} \in \mathbb{Z}^+$: Amount of ingredient $i$ used to create brew $b$

#### Model

$$ \max \sum_{i \in b} c_b y_p$$
such that:
$$y_b \leq x_{ib} \qquad \forall i \in I_b,  \forall b \in B$$
$$\sum_{\{b \mid i \in I_b\}} x_{ib} \leq z_i \qquad \forall i \in I$$


An additional sophistication for the model is to allow the purchase of common ingredients, which would amount to adding a integer decision variable for each such ingredient, adding them to the RHSs of the second constraints, and subtracting the costs in the objective.

In [5]:
B = [i for i in range(len(brews_df))]
I = [i for i in range(len(ingredients_df))]
c = np.array(brews_df['value'])

In [6]:
A = [[] for b in B]
for b in B:
    brew_ingredients = brews_df.loc[b][['first_ingredient','second_ingredient','third_ingredient']]
    for ingredient in brew_ingredients:
        if ingredient != 'NA':
            i = ingredients.index(ingredient)
            A[b].append(i)
            
# randomly generate inventory
z = np.random.poisson(lam=1, size=len(I))

Using Gurobi:

In [7]:
l = gp.tuplelist()
for b in B:
    for i in A[b]:
        l.append((b,i))
        
m = gp.Model('alchemy')
x = m.addVars(l, vtype=GRB.CONTINUOUS, name='x')
y = m.addVars(B, vtype=GRB.INTEGER, name='y')
m.addConstrs((y[b] <= x[b,i] for (b,i) in l), 'c1')
m.addConstrs((x.sum('*', i) <= z[i] for i in I))

m.setObjective(gp.quicksum(c[b]*y[b] for b in B), GRB.MAXIMIZE)
m.setParam('OUTPUT_FLAG',False)
m.optimize()

Academic license - for non-commercial use only - expires 2022-07-08
Using license file C:\Users\Eric\gurobi.lic


In [10]:
optimal_brews = [b for b in B if y[b].x > 0.5]
optimal_df = brews_df.loc[optimal_brews]
optimal_df['count'] = [int(y[b].x) for b in B if y[b].x > 0.5]
optimal_df.sort_values(by=['value'], ascending=False)
optimal_df = optimal_df[['name', 'count', 'value', 'first_ingredient', 'second_ingredient', 'third_ingredient', 'descriptions']]
optimal_df = optimal_df.sort_values(by='value', ascending=False)
optimal_df

,name,count,value,first_ingredient,second_ingredient,third_ingredient,descriptions
7126,Potion of Fortify Stamina,1,6588,Boar Tusk,Garlic,Slaughterfish Egg,[Stamina is increased by 46 points for 300 sec...
23091,Poison of Slow,1,5232,Large Antlers,Poison Bloom,Trama Root,[Carrying capacity increases by 29 for 300 sec...
3010,Potion of Fortify Health,1,5145,Bear Claws,Boar Tusk,Hanging Moss,[Decrease the target's Magicka regeneration by...
16694,Potion of Invisibility,1,824,Felsaad Tern Feathers,Luna Moth Wing,Vampire Dust,"[Cures all diseases., Increases Light Armor sk..."
8296,Poison of Damage Magicka Regen,1,820,Burnt Spriggan Wood,Chaurus Hunter Antennae,Salt Pile,[Decrease the target's Magicka regeneration by...
15183,Potion of Regenerate Health,1,804,Emperor Parasol Moss,Garlic,Troll Fat,"[Causes 22 points of poison damage., Two-hande..."
9257,Poison of Paralysis,1,701,Canis Root,Creep Cluster,Netch Jelly,[Carrying capacity increases by 29 for 300 sec...
9557,Poison of Paralysis,1,693,Canis Root,Rock Warbler Egg,Swamp Fungal Pod,"[Drain the target's Stamina by 28 points., One..."
13276,Potion of Regenerate Health,1,691,Daedra Heart,Juniper Berries,Namira's Rot,"[Drains the target's Magicka by 22 points., De..."
5249,Poison of Damage Magicka Regen,1,677,Blisterwort,Glow Dust,Human Heart,"[Drains the target's Magicka by 28 points., De..."


In [11]:
print(f"Optimal Objective Value: {int(m.objVal)}")

Optimal Objective Value: 32625
